In [ ]:
            ------------------------------------------------------ separate fwd and rev amplicon predictions --------------------------------------------------------------

In [1]:
import os
import pandas as pd
from Bio import SeqIO
import yaml
import numpy as np

# load in config with GPSC paths
with open("config/TB_allseqs.yaml", 'r') as file:
    config=yaml.safe_load(file)

# extract GPSCs
gpscs = config['samples'] 

# set params
amplicon_stats = list()
xlen = 2200

total_genome_coverages={}
amplicon_positions={}

with open('output.txt', 'w') as f:

    for gpsc, fasta_file in gpscs.items():
        records = list(SeqIO.parse(fasta_file, "fasta"))

        # calculate length of the genome for each GPSC
        genome_length=sum(len(record.seq) for record in records)

        # initialize amplicon genome coverage for each GPSC separately 
        total_genome_coverage=0

        # initialize set of covered positions across each GPSC 
        covered_positions = set()

        # intitialize list of amplicon positions 
        amplicon_positions[gpsc] = []

        print(f"Processing {gpsc}...", file=f)  

        # load the samtools depth file
        fwd_depth_file = os.path.join("samtools_depth_indiv_primers", f"{gpsc}_fwd.depth")
        rev_depth_file = os.path.join("samtools_depth_indiv_primers", f"{gpsc}_rev.depth")
        
        # load each depth file into a df
        fwd_df = pd.read_csv(fwd_depth_file, sep="\t", names=["Ref", "Pos", "Depth"])
        fwd_df['Primer'] = 'fwd'

        rev_df = pd.read_csv(rev_depth_file, sep="\t", names=["Ref", "Pos", "Depth"])
        rev_df['Primer'] = 'rev'

        # combine the dfs
        df = pd.concat([fwd_df, rev_df])

        # filter for fwd and rev positions separately 
        fwd_primer_binding_sites = df[(df["Depth"] == 1) & (df["Primer"] == 'fwd')]["Pos"].tolist()

        rev_primer_binding_sites = df[(df["Depth"] == 1) & (df["Primer"] == 'rev')]["Pos"].tolist()

        def calculate_amplicons(primer_binding_sites, gpsc, xlen, amplicon_positions, amplicon_stats, covered_positions, total_genome_coverage, f, primer_direction):
            for p1loc in primer_binding_sites:
        # find the next primer binding site within xlen bases
                p2loc = next((pos for pos in primer_binding_sites if p1loc < pos <= p1loc + xlen), None)

                if p2loc is not None:
            # amp stats
                    amplicon_stats.append((gpsc, gpsc, gpsc, gpsc, gpsc, gpsc, 0, 0, 0, p1loc, p2loc, primer_direction))
                    covered_positions.update(range(p1loc, p2loc+1))
            # ID amp positions
                    amplicon_positions[gpsc].append((p1loc, p2loc))
                    print(f"Detected amplicon from {p1loc} to {p2loc}.", file=f)  

            # get total genome coverage
                    total_genome_coverage += p2loc - p1loc

        # calculate predicted % coverage 
            coverage_percentage = (len(covered_positions) / genome_length) * 100

        # update dictionary 
            total_genome_coverages[gpsc] = coverage_percentage

        # summarize fwd amplicons
        calculate_amplicons(fwd_primer_binding_sites, gpsc, xlen, amplicon_positions, amplicon_stats, covered_positions, total_genome_coverage, f, 'fwd')
        # summarize rev amplicons
        calculate_amplicons(rev_primer_binding_sites, gpsc, xlen, amplicon_positions, amplicon_stats, covered_positions, total_genome_coverage, f, 'rev')

colnames = ["pid1", "pid2", "set1", "set2",
            "pseq1", "pseq2",
            "max_hdist", "hdist1", "hdist2",
            "p1loc", "p2loc", "PrimerDirection"]
coltypes = ["<U30", "<U30", "<U30", "<U30",
            "<U30", "<U30",
            float, float, float,
            int, int, "<U3"]

dt = {'names': colnames, 'formats': coltypes}

amplicon_statsnp = np.array(amplicon_stats,
                     dtype=dt)

np.save("amplicon_statstab.npy", amplicon_statsnp)

In [2]:
import csv

# Initialize a dictionary to group amplicons by sequence and primer type
grouped_amplicons = {}

# Define the maximum allowed size for amplicons
max_amplicon_size = 2000

# Iterate over amplicon_stats, assuming the last element is primer_direction, and the first is gpsc
for amplicon in amplicon_stats:
    *_, p1loc, p2loc, primer_direction = amplicon
    gpsc = amplicon[0]
    key = (gpsc, primer_direction)
    if key not in grouped_amplicons:
        grouped_amplicons[key] = []
    grouped_amplicons[key].append((p1loc, p2loc))

# Initialize list for contiguous amplicons
contiguous_amplicons = []

# Process each group for contiguity
for (gpsc, primer_direction), positions in grouped_amplicons.items():
    positions.sort()
    start, end = positions[0]
    
    for current_start, current_end in positions[1:]:
        # Check if contiguous and does not exceed max amplicon size
        if current_start <= end + 1 and (current_end - start) <= max_amplicon_size:
            end = max(end, current_end)
        else:
            contiguous_amplicons.append([gpsc, start, end, primer_direction])
            start, end = current_start, current_end
    # After processing all positions, append the last contiguous segment
    contiguous_amplicons.append([gpsc, start, end, primer_direction])

# Write contiguous amplicons to CSV
with open('amplicon_positions.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Sequence", "Start", "End", "Primer Type"])
    for amplicon in contiguous_amplicons:
        csvwriter.writerow(amplicon)

In [3]:
import pandas as pd

# create csv file containing predicted genome coverage for each sequence based on predicted amplicon coverage
total_genome_coverages_df = pd.DataFrame([total_genome_coverages])

total_genome_coverages_df.to_csv('genome_coverage_pc.csv', index=False)

In [4]:
for gpsc, coverage in total_genome_coverages.items():
    print(f"{gpsc}: {coverage}%")

ERR025455: 94.30636926995331%
ERR229952: 94.24949749182284%
ERR553116: 94.30500532355832%
SRR1166201: 94.22758521887519%
SRR1183039: 94.23123392552824%
ERR067660: 94.26439912936287%
ERR234150: 94.22438211884861%
H37Rv: 94.29513375398841%
SRR1166327: 94.2637081204518%
SRR1183123: 94.23559277888386%
ERR108423: 94.26489709538602%
ERR234176: 94.22227964769473%
H37Rv.fasta: 94.29513375398841%
SRR1167069: 94.2597631222043%
SRR1184373: 94.26377707687%
ERR108430: 94.2633158775919%
ERR234181: 94.25937482361533%
NC_019950: 89.43826698968014%
SRR1172044: 94.2951933896527%
SRR671749: 94.29532559441637%
ERR133985: 94.22833241678904%
ERR234202: 94.25072140831597%
SRR1172130: 94.29527356473999%
SRR671751: 94.29505677220168%
ERR137193: 94.20457041536598%
ERR234234: 94.25940693799221%
SRR1003103: 94.29564020529801%
SRR1172231: 94.26355538337981%
SRR671753: 94.26361948629834%
ERR144577: 94.26547606957885%
ERR234250: 94.29513504715933%
SRR1013560: 94.26350966153734%
SRR1172856: 94.2604850007027%
SRR67176

In [5]:
from Bio import SeqIO

# Load the config file
with open("config/TB_allseqs.yaml", 'r') as file:
    config=yaml.safe_load(file)

# Extract the GPSCs
gpscs = config['samples']

# Initialize a dictionary to store the genome lengths
genome_lengths = {}

# Calculate and store the length of each sequence
for gpsc, fasta_file in gpscs.items():
    records = list(SeqIO.parse(fasta_file, "fasta"))
    genome_length = sum(len(record.seq) for record in records)
    genome_lengths[gpsc] = genome_length  # Store the genome length in the dictionary

#Print the genome lengths
for gpsc, length in genome_lengths.items():
    print(f"{gpsc}: {length} bp")

ERR025455: 4411526 bp
ERR229952: 4411371 bp
ERR553116: 4411523 bp
SRR1166201: 4411464 bp
SRR1183039: 4411498 bp
ERR067660: 4411482 bp
ERR234150: 4411528 bp
H37Rv: 4411532 bp
SRR1166327: 4411491 bp
SRR1183123: 4411451 bp
ERR108423: 4411464 bp
ERR234176: 4411515 bp
H37Rv.fasta: 4411532 bp
SRR1167069: 4411473 bp
SRR1184373: 4411422 bp
ERR108430: 4411538 bp
ERR234181: 4411523 bp
NC_019950: 4432426 bp
SRR1172044: 4411508 bp
SRR671749: 4411470 bp
ERR133985: 4411498 bp
ERR234202: 4411510 bp
SRR1172130: 4411570 bp
SRR671751: 4411490 bp
ERR137193: 4411459 bp
ERR234234: 4411478 bp
SRR1003103: 4411538 bp
SRR1172231: 4411513 bp
SRR671753: 4411510 bp
ERR144577: 4411456 bp
ERR234250: 4411533 bp
SRR1013560: 4411565 bp
SRR1172856: 4411540 bp
SRR671760: 4411466 bp
ERR161044: 4411406 bp
ERR234256: 4411517 bp
SRR1013572: 4411459 bp
SRR1173388: 4411575 bp
SRR671767: 4411464 bp
ERR163962: 4411508 bp
ERR234257: 4411553 bp
SRR1019139: 4411483 bp
SRR1173445: 4411536 bp
SRR671790: 4411477 bp
ERR164005: 4411573

In [6]:
import csv
import yaml
from Bio import SeqIO

# Load the config file
with open("config/TB_allseqs.yaml", 'r') as file:
    config = yaml.safe_load(file)

# Extract the GPSCs
gpscs = config['samples']

# Open a CSV file to write
with open('assembly_lengths.csv', mode='w', newline='') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)
    
    # Calculate and store the length of each sequence, then write to CSV
    for gpsc, fasta_file in gpscs.items():
        records = list(SeqIO.parse(fasta_file, "fasta"))
        genome_length = sum(len(record.seq) for record in records)
        # Write the ID, start (0), and the length to the CSV file
        csv_writer.writerow([gpsc, 0, genome_length])